# 🌐 PGR Data Loader – Four-Phase Workflow

## 📘 Overview
This notebook follows a streamlined **four-phase MDMS data upload workflow**, required for onboarding any new State/ULB into PGR and Common Services.

---

# 🟦 PHASE 1: Tenant + Branding Setup
- Upload **Tenant + Branding + Basic Admin structure**
- Only **4 mandatory tenant fields**
- Automatically generates:
  - Tenant information  
  - Branding configuration  
  - Basic localization keys  

---

# 🟩 PHASE 2: Boundary Setup
- Upload complete **boundary hierarchy**
- Upload **boundary tree** (district → region → ward → block…)
- Supports custom boundary levels (Level 1–5)
- Automatically maps boundaries to the tenant

---

# 🟪 PHASE 3: Common & Complaint Master
Configure complete PGR functional data:
- Departments  
- Designations  
- Complaint Types  
- Complaint Sub-Types  
- SLA Hours  
- Localization entries  

Automatically generates:
- Department codes  
- Designation codes  
- Complaint type IDs  
- Localization message codes


---

# 🟦 PHASE 4: Employee Bulk Creation (NEW!)
Bulk create employees for your tenant:
- **Dynamic template generation** - Fetches departments, designations, roles from MDMS
- **Name-based Excel** - Use names instead of codes (auto-converted)
- **Auto role validation** - Missing roles automatically created in MDMS
- **Status tracking** - Color-coded results (🟢 SUCCESS, 🟡 EXISTS, 🔴 FAILED)
- **README - Roles sheet** - Copy-paste ready role names

Automatically handles:
- Employee code generation
- Role validation & creation
- Jurisdiction setup
- User account creation
- Error tracking with Excel updates

---

# ⚠️ IMPORTANT: Run Cells in Correct Order

To avoid errors, always run the notebook **top → bottom**:

1. **Initialize Global Variables** (Cell 2)  
2. **Import Dependencies** (Cell 3)  
3. **Download Templates** (Cell 4)  
4. **Proceed to Phase 1 → Phase 2 → Phase 3 → Phase 4**

❌ Do **NOT** skip cells  
❌ Do **NOT** jump directly to later phases  
✔ Always complete phases sequentially

---

# 📁 Templates Required

| Template File | Purpose |
|---------------|---------||
| **Tenant And Branding Master.xlsx** | Phase 1: Tenant Info + Branding |
| **Boundary Master.xlsx** | Phase 2: Complete boundary hierarchy + tree |
| **Common and Complaint Master.xlsx** | Phase 3: Departments, Designations, Complaints |
| **Employee_Master_Dynamic_{tenant}.xlsx** | Phase 4: Bulk employee creation (auto-generated) |

---

# 📝 Prerequisites

- State tenant ID (e.g., `pg`, `pb`, `mh`)
- Templates stored inside the `/templates` folder
- Phase 1-3 completed before Phase 4 (departments, designations, boundaries must exist)

---



In [111]:
!pip install notebook jupyterlab ipykernel ipywidgets pandas openpyxl xlsxwriter tqdm requests python-dotenv sqlalchemy psycopg2-binary plotly
!jupyter nbextension enable --py widgetsnbextension


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [112]:
import os
from IPython.display import display, HTML

templates_path = "templates"

template_files = [
    ("Tenant And Branding Master.xlsx", "PHASE 1: Tenant Info (4 fields) + Branding"),
    ("Common and Complaint Master.xlsx", "PHASE 3: Departments, Designations & Complaint Types"),
]

html_links = []
for i, (filename, description) in enumerate(template_files, 1):
    filepath = os.path.join(templates_path, filename)
    if os.path.exists(filepath):
        html_links.append(
            f'''
            <div style="margin: 8px 0;">
                <b>{i}.</b>
                <a href="{filepath}" download="{filename}"
                   style="font-size: 14px; color: #0066cc; text-decoration: none;">
                    {filename}
                </a>
                <span style="color: #666;"> - {description}</span>
            </div>
            '''
        )
    else:
        html_links.append(
            f'''
            <div style="margin: 8px 0;">
                <b>{i}.</b> {filename}
                <span style="color: red;">(Not found)</span>
            </div>
            '''
        )

html_content = f"""
<div style="border: 1px solid #ddd; padding: 15px; border-radius: 6px; background-color: #f9f9f9;">
    <h3 style="margin-top: 0;">📁 MDMS  Templates</h3>

    {''.join(html_links)}

    <hr style="margin: 18px 0;">

    <p><b>🧭 4-Phase Workflow:</b></p>
    <ol>
        <li><b>PHASE 1:</b> Tenant + Branding </li>
        <li><b>PHASE 2:</b> Boundary Master (Auto Generated based on userInput)</li>
        <li><b>PHASE 3:</b> Departments, Designations & Complaint Types</li>
            <li><b>PHASE 4:</b> Employee Master (Auto Generated based on userInput)</li>
    </ol>

  
</div>
"""

display(HTML(html_content))


In [113]:
# Import required packages
import pandas as pd
import json
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, FileLink
from unified_loader import UnifiedExcelReader, APIUploader, clean_nans
import shutil
import os
import sys
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Initialize Global Variables
CONFIG = {}
CONFIG_SET = False
TENANT_FILE = None
COMMON_MASTER_FILE = None
UPLOADED_TENANTS = []
SELECTED_TENANT = None
UPLOADER = None  # Global authenticated uploader instance

# Force reload modules to get latest changes
if 'unified_loader' in sys.modules:
    del sys.modules['unified_loader']
if 'mdms_validator' in sys.modules:
    del sys.modules['mdms_validator']
if 'excel_validator' in sys.modules:
    del sys.modules['excel_validator']

# Clear upload folder
upload_dir = 'upload'
if os.path.exists(upload_dir):
    for filename in os.listdir(upload_dir):
        file_path = os.path.join(upload_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
    print(f"✅ Cleared all files from {upload_dir}/")

print("✅ Packages loaded successfully!")
print("✅ Modules reloaded with latest changes!")
print()
print("="*70)
print("  ⚙️ AUTHENTICATION REQUIRED")
print("="*70)
print("Please proceed to the next cell to authenticate with the gateway.")

✅ Cleared all files from upload/
✅ Packages loaded successfully!
✅ Modules reloaded with latest changes!

  ⚙️ AUTHENTICATION REQUIRED
Please proceed to the next cell to authenticate with the gateway.


In [114]:
print("="*70)
print("        🔐 GATEWAY AUTHENTICATION")
print("="*70)
print()

# Authentication widgets
base_url_input = widgets.Text(
    value="",
    description='🌐 Gateway URL:',
    placeholder='e.g., https://unified-dev.digit.org',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

username_input = widgets.Text(
    value="",
    description='👤 Username:',
    placeholder='e.g., DEV_SUPER_ADMIN',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

password_input = widgets.Password(
    value="",
    description='🔑 Password:',
    placeholder='Enter password',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

user_type_input = widgets.Dropdown(
    options=['EMPLOYEE'],
    value="EMPLOYEE",
    description='👔 User Type:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

tenant_id_input = widgets.Text(
    value="",
    description='🏛️ Tenant ID:',
    placeholder='e.g., dev, pg, pb',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

auth_button = widgets.Button(
    description='🔐 Authenticate',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

auth_output = widgets.Output()

def on_authenticate(b):
    global UPLOADER, CONFIG
    
    with auth_output:
        clear_output()
        
        # Validate inputs
        if not base_url_input.value.strip():
            print("❌ Gateway URL is required")
            return
        
        if not username_input.value.strip():
            print("❌ Username is required")
            return
        
        if not password_input.value.strip():
            print("❌ Password is required")
            return
        
        if not tenant_id_input.value.strip():
            print("❌ Tenant ID is required")
            return
        
        print("🔄 Authenticating with gateway...")
        print(f"   Gateway: {base_url_input.value}")
        print(f"   Username: {username_input.value}")
        print(f"   User Type: {user_type_input.value}")
        print(f"   Tenant ID: {tenant_id_input.value}")
        print()
        
        try:
            # Create APIUploader with authentication
            UPLOADER = APIUploader(
                base_url=base_url_input.value.strip(),
                username=username_input.value.strip(),
                password=password_input.value.strip(),
                user_type=user_type_input.value,
                tenant_id=tenant_id_input.value.strip()
            )
            
            if UPLOADER.authenticated:
                # Save config
                CONFIG = {
                    'base_url': base_url_input.value.strip(),
                    'tenant_id': tenant_id_input.value.strip()
                }
                CONFIG_SET = True
                
                print()
                print("="*70)
                print("  ✅ AUTHENTICATION SUCCESSFUL!")
                print("="*70)
                print(f"\n👤 Logged in as: {UPLOADER.user_info.get('userName', 'Unknown')}")
                print(f"📧 Email: {UPLOADER.user_info.get('emailId', 'N/A')}")
                print(f"🏛️ Tenant: {UPLOADER.user_info.get('tenantId', 'N/A')}")
                print(f"\n🎭 Roles:")
                for role in UPLOADER.user_info.get('roles', []):
                    print(f"   • {role.get('name', 'Unknown')} ({role.get('code', 'N/A')})")
                print()
                print("="*70)
                print("  ➡️ Proceed to PHASE 1: Tenant Setup")
                print("="*70)
            else:
                print("❌ Authentication failed. Please check your credentials.")
        
        except Exception as e:
            print(f"❌ Error during authentication: {str(e)}")

auth_button.on_click(on_authenticate)

# Display UI
auth_ui = widgets.VBox([
    widgets.HTML("<h3>🔐 Step 0: Gateway Authentication</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Authenticate with the eGov gateway to access all services</i></p>"),
    widgets.HTML("<br>"),
    base_url_input,
    widgets.HTML("<p style='font-size: 11px; color: #888;'><i>💡 Gateway URL - same for all services</i></p>"),
    widgets.HTML("<br>"),
    username_input,
    password_input,
    user_type_input,
    tenant_id_input,
    widgets.HTML("<p style='font-size: 11px; color: #ff6600;'><i>⚠️ Credentials are used only for this session and not stored</i></p>"),
    widgets.HTML("<br>"),
    auth_button,
    auth_output
])

display(auth_ui)

        🔐 GATEWAY AUTHENTICATION



In [115]:
import ipywidgets as widgets
from IPython.display import display, clear_output



# Initialize uploader
uploader = UPLOADER
if uploader is None:
    display(widgets.HTML(
        "<div style='background:#fff4cd;padding:10px;border-radius:5px;'>"
        "<b>⚠ Authenticate in order to proceed.</b></div>"
    ))
    raise SystemExit
else:
    tenants = uploader.search_mdms_data(schema_code='tenant.tenants', tenant='pg')
    
    

# Fetch existing tenants


tenant_codes = [t["code"] for t in tenants] if tenants else []


    

# Title
title = widgets.HTML("<h3 style='color:#444;margin-bottom:10px;'>⚙️ Tenant Configuration</h3>")

# Info box
if tenants:
    tenant_list_html = "<br>".join([f"• <b>{t['code']}</b> - {t['name']}" for t in tenants])
    info_box = widgets.HTML(
        f"<div style='background:#e7f3ff;padding:10px;border-radius:5px;'>"
        f"<b>✔ Found {len(tenants)} existing tenant(s) in MDMS:</b><br>{tenant_list_html}</div>"
    )
else:
    info_box = widgets.HTML(
        "<div style='background:#fff3cd;padding:10px;border-radius:5px;'>"
        "<b>⚠ No existing tenants found in MDMS.</b></div>"
    )

# YES/NO question
question = widgets.HTML("<p><b>Do you want to use an existing tenant?</b></p>")

yes_no_buttons = widgets.ToggleButtons(
    options=[('YES - Use Existing', 'yes'), ('NO - Create New', 'no')],
)

# Existing tenant dropdown
existing_dropdown = widgets.Dropdown(
    options=tenant_codes,
    description='Select Tenant:',
    style={'description_width': '120px'},
    layout=widgets.Layout(visibility='hidden')
)

# New tenant UI (each field hidden individually)
new_label = widgets.HTML(
    "<p style='color:#666;font-size:12px;'><i>💡 Fill in details for the new tenant</i></p>",
    layout=widgets.Layout(visibility='hidden')
)

new_tenant_code = widgets.Text(
    placeholder='e.g., pg.cityc, pb.amritsar',
    description='Tenant Code:',
    style={'description_width': '120px'},
    layout=widgets.Layout(visibility='hidden')
)




# Proceed button
proceed_button = widgets.Button(
    description="✅ Proceed",
    button_style='success',
    icon='check',
    layout=widgets.Layout(visibility='hidden')
)

output = widgets.Output()

# ----- VISIBILITY TOGGLE -----
def on_yes_no_change(change):
    choice = change['new']

    if choice == 'yes':
        existing_dropdown.layout.visibility = 'visible'

        # Hide new tenant input fields
        for w in [
            new_tenant_code
        ]:
            w.layout.visibility = 'hidden'

        proceed_button.layout.visibility = 'visible'

    elif choice == 'no':
        existing_dropdown.layout.visibility = 'hidden'

        # Show new tenant input fields
        for w in [
            new_tenant_code
        ]:
            w.layout.visibility = 'visible'

        proceed_button.layout.visibility = 'visible'

yes_no_buttons.observe(on_yes_no_change, names='value')

# ----- PROCEED BUTTON LOGIC -----
def on_proceed_clicked(b):
    with output:
        clear_output()

        # Existing tenant selected
        if yes_no_buttons.value == 'yes':
            selected = existing_dropdown.value
            print(f"✅ Proceeding with existing tenant: {selected}")
            return

        # New tenant flow
        tenant_code = new_tenant_code.value.strip()
        

        # Validation
        if not tenant_code:
            print("❌ Tenant code is required")
            return
      

        print(f"🔄 Creating new tenant: {tenant_code}...\n")

        # ---- CORRECT API CALL ----
        result = uploader.setup_default_data(
            targetTenantId=tenant_code,
            module="tenant",
            schemaCodes=[
              "tenant.citymodule",
              "tenant.tenants"
            ],
            onlySchemas=False
        )

        if result.get("success"):
            print("✅ Tenant created successfully!")
            print(f"➡ Next: Proceed to Phase 2 for tenant: {tenant_code}")
        else:
            print("❌ Failed to create tenant")
            print(result)

proceed_button.on_click(on_proceed_clicked)

# UI Layout
ui = widgets.VBox([
    title,
    info_box,
    question,
    yes_no_buttons,
    existing_dropdown,
    new_label,
    new_tenant_code,
    proceed_button,
    output
])

display(ui)


HTML(value="<div style='background:#fff4cd;padding:10px;border-radius:5px;'><b>⚠ Authenticate in order to proc…

SystemExit: 

---

# 🟦 PHASE 1: TENANT/STATE SETUP

## 📋 Overview
In Phase 1, you will set up the foundational tenant and branding configuration for your state/ULB.

## 🎯 What You'll Do:
1. **Enter State Tenant ID** (e.g., `pg` for Punjab, `pb` for Punjab, `mh` for Maharashtra)
2. **Upload Tenant Master Excel** with state and city information
3. **Upload tenant data** - Creates tenant records in MDMS
4. **Upload state branding** - Logo, colors, theme configuration

## 📄 Template Required:
- **Tenant And Branding Master.xlsx**

## ✅ What Gets Created:
- Tenant information in MDMS
- City-level tenant records (e.g., `pg.citya`)
- Branding configuration (logo, theme, colors)
- Basic localization keys
- Admin boundary structure

## ⚠️ Important:
- Make sure the Excel file is in the `templates/` folder
- Tenant ID must be lowercase (e.g., `pg`, not `PG`)
- Complete Phase 1 before moving to Phase 2


In [110]:
# Configuration Setup and Tenant Master Upload
# Make sure you've run the "Initialize Global Variables" cell first!

print("="*70)
print("        ⚙️  PHASE 1: CONFIGURATION & UPLOAD TENANT MASTER")
print("="*70)
print()

# Configuration widgets
state_tenant_id = widgets.Text(
    value='pg',
    description='🏛️ State Tenant ID:',
    placeholder='e.g., pg, pb, up',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# File upload widget
tenant_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

# Combined button
upload_btn = widgets.Button(
    description='✅ Save Config & Upload Tenant Master',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

output = widgets.Output()

def on_upload(b):
    global CONFIG, CONFIG_SET, TENANT_FILE, UPLOADED_TENANTS
    
    with output:
        clear_output()
        
        # Validate configuration
        if not state_tenant_id.value.strip():
            print("❌ State Tenant ID required")
            return
        
        # Validate file selection
        if not tenant_file_upload.value:
            print("❌ Please select Tenant Master Excel file")
            return
        
        # Save configuration
        CONFIG = {
            'base_url': 'https://unified-qa.digit.org',
            'state_tenant_id': state_tenant_id.value.strip().lower()
        }
        CONFIG_SET = True
        
        print("="*70)
        print("  ✅ CONFIGURATION SAVED!")
        print("="*70)
        print(f"\n📊 Configuration:")
        print(f"   • State Tenant: {CONFIG['state_tenant_id']}")
        print("="*70)
        
        # Save file
        os.makedirs('upload', exist_ok=True)
        uploaded_file = tenant_file_upload.value[0]
        content = uploaded_file['content']
        filename = 'Tenant_Master.xlsx'
        
        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)
        
        print("\n" + "="*70)
        print("  📋 VALIDATING FILE AGAINST MDMS SCHEMA...")
        print("="*70)
        
        # Validate the file against MDMS schema
        try:
            from mdms_validator import MDMSValidator
            mdms_validator = MDMSValidator(
                base_url=UPLOADER.base_url,
                auth_token=UPLOADER.auth_token,
                user_info=UPLOADER.user_info
            )
            result = mdms_validator.validate_excel_file(
                excel_file=upload_path,
                tenant_id=CONFIG['state_tenant_id'],
                schema_code='tenant.master'
            )
            
            if result['valid']:
                print("\n✅ MDMS VALIDATION PASSED!")
                print("   All required fields match MDMS schema.")
                TENANT_FILE = upload_path
                
                print("\n" + "="*70)
                print("  ✅ FILE UPLOADED SUCCESSFULLY")
                print("="*70)
                print(f"\n📄 File: {upload_path}")
                print("\n➡️  Proceed to load and upload tenant data")
                print("="*70)
            else:
                print("\n❌ MDMS VALIDATION FAILED!")
                print(f"   Found {len(result['errors'])} error(s):\n")
                
                # Show first 5 errors
                for i, error in enumerate(result['errors'][:5], 1):
                    print(f"   Error {i}:")
                    if 'row' in error:
                        print(f"      Sheet: {error.get('sheet', 'Unknown')}")
                        print(f"      Row: {error['row']}")
                        if 'column' in error:
                            print(f"      Column: {error['column']}")
                        if 'value' in error:
                            print(f"      Value: {error.get('value', 'N/A')}")
                    print(f"      Message: {error['message']}")
                    print()
                
                if len(result['errors']) > 5:
                    print(f"   ... and {len(result['errors']) - 5} more errors")
                
                print("\n⚠️  Please fix the errors in your Excel file and try again.")
                print("="*70)
                TENANT_FILE = None
                CONFIG_SET = False
                
        except Exception as e:
            print(f"\n⚠️  MDMS Validation error: {str(e)}")
            print("   Proceeding without validation...")
            TENANT_FILE = upload_path
            print("\n" + "="*70)
            print("  ✅ FILE UPLOADED SUCCESSFULLY")
            print("="*70)
            print(f"\n📄 File: {upload_path}")
            print("\n➡️  Proceed to load and upload tenant data")
            print("="*70)

upload_btn.on_click(on_upload)

# Display
combined_box = widgets.VBox([
    widgets.HTML("<h3>📋 Step 1.1: Configuration & Upload Tenant Master</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Enter your state tenant ID and upload the Tenant Master Excel file</i></p>"),
    widgets.HTML("<p style='color: #666;'><b>Example:</b> If your state is Punjab, enter <code>pg</code></p>"),
    widgets.HTML("<br>"),
    state_tenant_id,
    widgets.HTML("<p style='font-size: 12px; color: #888;'><i>💡 The Tenant Master Excel will contain cities like pg.citya, pg.cityb under this state tenant</i></p>"),
    widgets.HTML("<br>"),
    widgets.HTML("<p style='color: #666;'><b>Select Tenant Master Excel File:</b></p>"),
    tenant_file_upload,
    widgets.HTML("<p style='font-size: 11px; color: #007bff;'><i>⚡ File will be validated against MDMS schema 'tenant.master'</i></p>"),
    widgets.HTML("<br>"),
    upload_btn,
    output
])

display(combined_box)

        ⚙️  PHASE 1: CONFIGURATION & UPLOAD TENANT MASTER



---

## 📤 Step 1.1: Upload Tenant Master Excel

**Instructions:**
1. Click the **Upload** button below
2. Select **Tenant And Branding Master.xlsx** from your computer
3. The file will be uploaded to the `templates/` folder
4. You'll see a confirmation message

**What happens next:**
- System reads tenant and city information from Excel
- Creates tenant records in MDMS
- Sets up basic admin structure

💡 **Tip**: Make sure the Excel file has the correct format with State ID and City details.


In [102]:
print("="*70)
print("[PHASE 1] LOADING TENANT DATA")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")


if not TENANT_FILE:
    print("❌ Please upload Tenant Master Excel first!")
else:
    # Initialize reader and uploader
    reader = UnifiedExcelReader(TENANT_FILE)
    uploader = UPLOADER
    
    # Read tenant data using NEW format method
    tenants_data, tenants_localization = reader.read_tenant_info()
    
    print(f"\n[INFO] Loaded {len(tenants_data)} tenant(s) from Excel")
    for tenant in tenants_data:
        print(f"   - {tenant['code']}: {tenant['name']} ({tenant['type']})")
        UPLOADED_TENANTS.append(tenant['code'])
    
    print(f"\n[INFO] Auto-generated {len(tenants_localization)} localization entries")
    
    # Upload Tenants
    result_tenants = uploader.create_mdms_data(
    schema_code='tenant.tenants',
    data_list=clean_nans(tenants_data),
    tenant=CONFIG['state_tenant_id'],
    sheet_name='Tenant Info',  # Exact sheet name from your Excel
    excel_file=TENANT_FILE      # ← Add this
    )
    
    # Upload Localization
    result_loc = uploader.create_localization_messages(
        localization_list=clean_nans(tenants_localization),
        tenant=CONFIG['state_tenant_id'],
        sheet_name='Tenants_Localization'
    )
    print(result_tenants)
    # Summary
    if result_tenants['failed'] == 0:
        print("\n✅ [SUCCESS] Tenants uploaded successfully!")
        print("\n➡️  Proceed to upload State Branding (optional)")
    else:
        print("\n⚠️  [WARNING] Some tenants failed. Check errors/ folder.")

[PHASE 1] LOADING TENANT DATA
❌ Please upload Tenant Master Excel first!


---

## 🎨 Step 1.2: Upload State Branding Configuration

**Instructions:**
1. Run the cell below to process branding data
2. System reads the **State Branding** sheet from Excel
3. Uploads logo, colors, and theme configuration

**What gets configured:**
- State/ULB logo
- Primary and secondary colors
- Login page theme
- Header/footer branding

**Expected Result:**
- ✅ Branding data uploaded to MDMS
- ✅ Logo and theme visible in PGR application

💡 **Tip**: After this step, Phase 1 is complete. Proceed to Phase 2 for boundary setup.


In [103]:
print("="*70)
print("[PHASE 1] LOADING STATE BRANDING DATA")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")


if not TENANT_FILE:
    print("❌ Please upload Tenant Master Excel first!")
else:
    # Initialize reader and uploader
    reader = UnifiedExcelReader(TENANT_FILE)
    uploader = UPLOADER
    
    # Read branding data using NEW format method
    branding_data = reader.read_tenant_branding(CONFIG['state_tenant_id'])
    
    if not branding_data:
        print("\n[INFO] No branding data found in 'Tenant Branding Deatils' sheet")
        print("   This is optional - you can skip this step if not needed")
    else:
        print(f"\n[INFO] Loaded {len(branding_data)} branding record(s)")
        for brand in branding_data:
            print(f"   - {brand['code']}")
        
        # Upload Branding to tenant.stateInfo schema
        result_branding = uploader.create_mdms_data(
            schema_code="common-masters.StateInfo",
            data_list=clean_nans(branding_data),
            tenant=CONFIG['state_tenant_id'],
            sheet_name='Tenant Branding Deatils',  # Exact sheet name from your Excel
            excel_file=TENANT_FILE      # ← Add this
        )
        
        # Summary
        if result_branding['failed'] == 0:
            print("\n✅ [SUCCESS] State Branding uploaded successfully!")
            print("\n🎉 PHASE 1 COMPLETED!")
            print("\n➡️  Proceed to PHASE 2: Common Master Data Upload")
        else:
            print("\n⚠️  [WARNING] Some branding records failed. Check errors/ folder.")

[PHASE 1] LOADING STATE BRANDING DATA
❌ Please upload Tenant Master Excel first!


# Phase 2: Boundary Master Data

---

# 🟩 PHASE 2: BOUNDARY MANAGEMENT WORKFLOW

This phase implements the complete boundary management workflow with template generation.

## 📋 Workflow Steps:

### **Option 1: Create New Hierarchy (First Time)**
1. Enter tenant ID and hierarchy type
2. Define boundary levels (City → Zone → Block → Locality)
3. Create the hierarchy definition
4. Generate and download Excel template
5. Fill template with boundary data
6. Upload and process filled template

### **Option 2: Use Existing Hierarchy**
1. Search for existing hierarchies
2. Select hierarchy and generate template
3. Download, fill, upload template

## 🎯 What You Need:
- **Tenant ID** (e.g., `pg`, `mz`, `pb`)
- **Hierarchy Type** (e.g., `ADMIN`, `ADMIN1`, `REVENUE`)
- **Boundary Levels** (e.g., City, Zone, Block, Locality)
- **Boundary data** to fill in the downloaded template

---

In [78]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from unified_loader import APIUploader
import time
import os

print("="*70)
print("   🔍 PHASE 2.1: BOUNDARY HIERARCHY & TEMPLATE GENERATION")
print("="*70)

# Global variables
BOUNDARY_TENANT = None
BOUNDARY_HIERARCHY_TYPE = None
TEMPLATE_FILESTORE_ID = None
DOWNLOADED_TEMPLATE_PATH = None
DOWNLOAD_URL = None


if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")

uploader = UPLOADER

# UI Widgets
tenant_input = widgets.Text(
    value='pg',
    description='🏛️ Tenant ID:',
    placeholder='e.g., pg, mz, pb',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

tab_selector = widgets.Tab()

# TAB 1: CREATE NEW HIERARCHY
hierarchy_type_input = widgets.Text(
    value='ADMIN',
    description='Hierarchy Type:',
    placeholder='e.g., ADMIN, ADMIN1',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

level_widgets = []
levels_container = widgets.VBox([])

add_level_btn = widgets.Button(
    description='➕ Add Level',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

create_hierarchy_btn = widgets.Button(
    description='🏗️ Create Hierarchy',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

generate_template_btn = widgets.Button(
    description='📄 Generate Template',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

download_template_btn = widgets.Button(
    description='📥 Download Template',
    button_style='info',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

create_output = widgets.Output()

# TAB 2: SEARCH EXISTING
search_btn = widgets.Button(
    description='🔍 Search Hierarchies',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px')
)

hierarchy_dropdown = widgets.Dropdown(
    options=[],
    description='Hierarchy:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%', visibility='hidden')
)

search_generate_btn = widgets.Button(
    description='📄 Generate Template',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

search_download_btn = widgets.Button(
    description='📥 Download Template',
    button_style='info',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

search_output = widgets.Output()

# Helper functions
def add_initial_level():
    global level_widgets
    level_widget = widgets.Text(
        value='',
        description='Level 1:',
        placeholder='e.g., City',
        style={'description_width': '140px'},
        layout=widgets.Layout(width='95%')
    )
    level_widgets.append(level_widget)
    levels_container.children = tuple(level_widgets)

def on_add_level_clicked(b):
    global level_widgets
    level_num = len(level_widgets) + 1
    level_widget = widgets.Text(
        value='',
        description=f'Level {level_num}:',
        placeholder=f'e.g., Zone, Block',
        style={'description_width': '140px'},
        layout=widgets.Layout(width='95%')
    )
    level_widgets.append(level_widget)
    levels_container.children = tuple(level_widgets)

add_level_btn.on_click(on_add_level_clicked)

def on_create_hierarchy_clicked(b):
    global BOUNDARY_TENANT, BOUNDARY_HIERARCHY_TYPE

    with create_output:
        clear_output()

        tenant = tenant_input.value.strip()
        if not tenant:
            print("❌ Please enter a tenant ID")
            return

        hierarchy_type = hierarchy_type_input.value.strip()
        if not hierarchy_type:
            print("❌ Please enter a hierarchy type")
            return

        levels = [lw.value.strip() for lw in level_widgets if lw.value.strip()]
        if len(levels) == 0:
            print("❌ Please add at least one boundary level")
            return

        BOUNDARY_TENANT = tenant
        BOUNDARY_HIERARCHY_TYPE = hierarchy_type

        print(f"🏗️ Creating boundary hierarchy...")
        print(f"   Tenant: {tenant}")
        print(f"   Hierarchy Type: {hierarchy_type}")
        print(f"   Levels: {' → '.join(levels)}")

        boundary_items = []
        prev_level = None

        for level in levels:
            boundary_items.append({
                "boundaryType": level,
                "parentBoundaryType": prev_level,
                "active": True
            })
            prev_level = level

        hierarchy_payload = {
            "tenantId": tenant,
            "hierarchyType": hierarchy_type,
            "boundaryHierarchy": boundary_items
        }

        try:
            result = uploader.create_boundary_hierarchy(hierarchy_payload)
            if result:
                print("\n➡️  Now generate the Excel template")
                generate_template_btn.layout.visibility = 'visible'
        except Exception as e:
            print(f"\n❌ Failed: {str(e)[:200]}")

create_hierarchy_btn.on_click(on_create_hierarchy_clicked)

def on_generate_template_clicked(b):
    global TEMPLATE_FILESTORE_ID

    with create_output:
        clear_output(wait=True)

        if not BOUNDARY_TENANT or not BOUNDARY_HIERARCHY_TYPE:
            print("❌ Please create hierarchy first")
            return

        print(f"📄 Generating template...")

        result = uploader.generate_boundary_template(BOUNDARY_TENANT, BOUNDARY_HIERARCHY_TYPE)
        if not result:
            print("❌ Generation failed")
            return

        print(f"\n⏳ Waiting for completion...")
        resource = uploader.poll_boundary_template_status(
            BOUNDARY_TENANT, BOUNDARY_HIERARCHY_TYPE, max_attempts=30, delay=2
        )

        if resource and resource.get('status') == 'completed':
            TEMPLATE_FILESTORE_ID = resource.get('fileStoreid')
            print(f"\n✅ Template ready!")
            download_template_btn.layout.visibility = 'visible'
        else:
            print("\n❌ Failed or timed out")

generate_template_btn.on_click(on_generate_template_clicked)

def on_download_template_clicked(b):
    global DOWNLOADED_TEMPLATE_PATH, DOWNLOAD_URL

    with create_output:
        if not TEMPLATE_FILESTORE_ID:
            print("❌ No template available")
            return

        print(f"\n📥 Downloading...")

        result = uploader.download_boundary_template(
            tenant_id=BOUNDARY_TENANT,
            filestore_id=TEMPLATE_FILESTORE_ID,
            hierarchy_type=BOUNDARY_HIERARCHY_TYPE,
            return_url=True
        )

        if result:
            DOWNLOADED_TEMPLATE_PATH = result['path']
            DOWNLOAD_URL = result['url']
            
            print(f"\n🎉 Downloaded!")
            print(f"\n" + "="*70)
            
            # Display clickable download link
            download_link_html = f"""
            <div style='background:#e7f3ff; padding:15px; border-radius:8px; margin:10px 0;'>
                <h3 style='margin-top:0; color:#0066cc;'>📥 Template Ready!</h3>
                <p><b>Local Path:</b> {DOWNLOADED_TEMPLATE_PATH}</p>
                <a href='{DOWNLOAD_URL}' download='boundary_template.xlsx' 
                   style='display:inline-block; padding:12px 24px; background:#007bff; color:white; 
                          text-decoration:none; border-radius:6px; font-weight:bold; margin:10px 0;'>
                    ⬇️ Click Here to Download Template
                </a>
            </div>
            """
            display(HTML(download_link_html))
            
            print("📝 NEXT STEPS:")
            print("1. Click the download link above OR open file locally")
            print("2. Fill boundary data in Excel")
            print("3. Save the file")
            print("4. Proceed to Step 2.2 to upload")
            print("="*70)

download_template_btn.on_click(on_download_template_clicked)

# TAB 2 HANDLERS
def on_search_clicked(b):
    global BOUNDARY_TENANT

    with search_output:
        clear_output()

        tenant = tenant_input.value.strip()
        if not tenant:
            print("❌ Please enter tenant ID")
            return

        BOUNDARY_TENANT = tenant
        print(f"🔍 Searching hierarchies in: {tenant}")

        hierarchies = uploader.search_boundary_hierarchies(tenant)

        if hierarchies:
            hierarchy_options = [(f"{h['hierarchyType']} ({len(h.get('boundaryHierarchy', []))} levels)", h['hierarchyType']) for h in hierarchies]
            hierarchy_dropdown.options = hierarchy_options
            hierarchy_dropdown.layout.visibility = 'visible'
            search_generate_btn.layout.visibility = 'visible'
        else:
            print("\n⚠️  No hierarchies found")

search_btn.on_click(on_search_clicked)

def on_search_generate_clicked(b):
    global BOUNDARY_HIERARCHY_TYPE, TEMPLATE_FILESTORE_ID

    with search_output:
        clear_output(wait=True)

        hierarchy_type = hierarchy_dropdown.value
        if not hierarchy_type:
            print("❌ Please select hierarchy")
            return

        BOUNDARY_HIERARCHY_TYPE = hierarchy_type
        print(f"📄 Generating template for {hierarchy_type}...")

        result = uploader.generate_boundary_template(BOUNDARY_TENANT, hierarchy_type)
        if not result:
            return

        resource = uploader.poll_boundary_template_status(
            BOUNDARY_TENANT, hierarchy_type, max_attempts=30, delay=2
        )

        if resource and resource.get('status') == 'completed':
            TEMPLATE_FILESTORE_ID = resource.get('fileStoreid')
            print(f"\n✅ Ready!")
            search_download_btn.layout.visibility = 'visible'

search_generate_btn.on_click(on_search_generate_clicked)

def on_search_download_clicked(b):
    global DOWNLOADED_TEMPLATE_PATH, DOWNLOAD_URL

    with search_output:
        if not TEMPLATE_FILESTORE_ID:
            return

        result = uploader.download_boundary_template(
            tenant_id=BOUNDARY_TENANT,
            filestore_id=TEMPLATE_FILESTORE_ID,
            hierarchy_type=BOUNDARY_HIERARCHY_TYPE,
            return_url=True
        )

        if result:
            DOWNLOADED_TEMPLATE_PATH = result['path']
            DOWNLOAD_URL = result['url']
            
            # Display clickable download link
            download_link_html = f"""
            <div style='background:#e7f3ff; padding:15px; border-radius:8px; margin:10px 0;'>
                <h3 style='margin-top:0; color:#0066cc;'>📥 Template Ready!</h3>
                <a href='{DOWNLOAD_URL}' download='boundary_template.xlsx' 
                   style='display:inline-block; padding:12px 24px; background:#007bff; color:white; 
                          text-decoration:none; border-radius:6px; font-weight:bold;'>
                    ⬇️ Click to Download Template
                </a>
            </div>
            """
            display(HTML(download_link_html))
            print(f"\n✅ Downloaded! Fill template and proceed to Step 2.2")

search_download_btn.on_click(on_search_download_clicked)

# Build Tab 1
add_initial_level()

tab1_content = widgets.VBox([
    widgets.HTML("<p style='color:#666;'><i>Create new hierarchy from scratch</i></p>"),
    hierarchy_type_input,
    widgets.HTML("<br><p style='font-weight:bold;'>🏛️ Boundary Levels:</p>"),
    levels_container,
    add_level_btn,
    widgets.HTML("<br>"),
    create_hierarchy_btn,
    generate_template_btn,
    download_template_btn,
    widgets.HTML("<br>"),
    create_output
])

# Build Tab 2
tab2_content = widgets.VBox([
    widgets.HTML("<p style='color:#666;'><i>Use existing hierarchy</i></p>"),
    search_btn,
    widgets.HTML("<br>"),
    hierarchy_dropdown,
    search_generate_btn,
    search_download_btn,
    widgets.HTML("<br>"),
    search_output
])

tab_selector.children = [tab1_content, tab2_content]
tab_selector.set_title(0, '🏗️ Create New')
tab_selector.set_title(1, '🔍 Use Existing')

main_ui = widgets.VBox([
    widgets.HTML("<h3>🔍 Step 2.1: Boundary Hierarchy & Template</h3>"),
    tenant_input,
    widgets.HTML("<br>"),
    tab_selector
])

display(main_ui)

   🔍 PHASE 2.1: BOUNDARY HIERARCHY & TEMPLATE GENERATION


In [79]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from unified_loader import APIUploader
import os
import time

print("="*70)
print("   📤 PHASE 2.2: UPLOAD & PROCESS BOUNDARY DATA")
print("="*70)

UPLOADED_FILESTORE_ID = None
PROCESSED_FILESTORE_ID = None
PROCESSED_DOWNLOAD_URL = None


if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")

uploader = UPLOADER

boundary_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

upload_process_btn = widgets.Button(
    description='📤 Upload & Process',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

check_status_btn = widgets.Button(
    description='🔍 Check Processing Status',
    button_style='info',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

download_processed_btn = widgets.Button(
    description='📥 Download Processed File',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px', visibility='hidden')
)

output = widgets.Output()

def on_upload_process_clicked(b):
    global UPLOADED_FILESTORE_ID, PROCESSED_FILESTORE_ID, PROCESSED_DOWNLOAD_URL

    with output:
        clear_output()

        if not BOUNDARY_TENANT:
            print("❌ Complete Step 2.1 first")
            return

        if not BOUNDARY_HIERARCHY_TYPE:
            print("❌ No hierarchy type selected")
            return

        if not boundary_file_upload.value:
            print("❌ Select filled template")
            return

        os.makedirs('upload', exist_ok=True)
        uploaded_file = boundary_file_upload.value[0]
        content = uploaded_file['content']
        filename = f'boundary_filled_{BOUNDARY_TENANT}_{BOUNDARY_HIERARCHY_TYPE}.xlsx'

        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)

        print(f"📄 Saved: {filename}")

        print(f"\n📤 Uploading to filestore...")
        filestore_id = uploader.upload_file_to_filestore(
            file_path=upload_path,
            tenant_id=BOUNDARY_TENANT,
            module='HCM-ADMIN-CONSOLE'
        )

        if not filestore_id:
            print("❌ Upload failed")
            return

        UPLOADED_FILESTORE_ID = filestore_id

        print(f"\n⚙️ Processing...")
        result = uploader.process_boundary_data(
            tenant_id=BOUNDARY_TENANT,
            filestore_id=filestore_id,
            hierarchy_type=BOUNDARY_HIERARCHY_TYPE,
            action='create'
        )

        if result:
            status = result.get('status')
            processed_id = result.get('processedFileStoreId')

            print(f"\n✅ Submitted")
            print(f"   Status: {status}")

            if status == 'data-accepted':
                print("\n" + "="*70)
                print("🎉 PHASE 2 COMPLETED!")
                print("="*70)
                print("✅ Boundaries created")
                print("\n⏳ Processing in background...")
                print("   Click 'Check Status' after 10-20 seconds")
                print("="*70)
                check_status_btn.layout.visibility = 'visible'
            elif processed_id:
                PROCESSED_FILESTORE_ID = processed_id
                download_processed_btn.layout.visibility = 'visible'
                print("\n💡 Processed file ready for download")

upload_process_btn.on_click(on_upload_process_clicked)

def on_check_status_clicked(b):
    global PROCESSED_FILESTORE_ID, PROCESSED_DOWNLOAD_URL
    
    with output:
        print("\n🔍 Checking processing status...")
        
        # Poll the _process endpoint to get status
        boundary_mgmt_url = "http://localhost:8099"
        url = f"{boundary_mgmt_url}/boundary-management/v1/_process-search"
        
        user_info_copy = uploader.user_info.copy()
        user_info_copy['tenantId'] = BOUNDARY_TENANT
        
        payload = {
            "RequestInfo": {
                "apiId": "Rainmaker",
                "authToken": uploader.auth_token,
                "userInfo": user_info_copy,
                "msgId": f"{int(time.time() * 1000)}|en_IN",
                "plainAccessRequest": {}
            },
            "ResourceSearchCriteria": {
                "tenantId": BOUNDARY_TENANT,
                "hierarchyType": BOUNDARY_HIERARCHY_TYPE
            }
        }
        
        try:
            import requests
            response = requests.post(url, json=payload, headers={'Content-Type': 'application/json'})
            response.raise_for_status()
            data = response.json()
            
            resources = data.get('ResourceDetails', [])
            if resources:
                latest = resources[0]
                status = latest.get('status')
                processed_id = latest.get('processedFileStoreId')
                
                print(f"   Status: {status}")
                
                if processed_id:
                    PROCESSED_FILESTORE_ID = processed_id
                    print(f"   Processed File ID: {processed_id}")
                    
                    # Get download URL
                    filestore_url = "http://localhost:8009"
                    file_url = f"{filestore_url}/filestore/v1/files/url"
                    params = {"tenantId": BOUNDARY_TENANT, "fileStoreIds": processed_id}
                    
                    file_response = requests.get(file_url, params=params)
                    file_response.raise_for_status()
                    file_data = file_response.json()
                    
                    file_urls = file_data.get('fileStoreIds', [])
                    if file_urls:
                        PROCESSED_DOWNLOAD_URL = file_urls[0].get('url')
                        
                        download_link_html = f"""
                        <div style='background:#d4edda; padding:15px; border-radius:8px; margin:10px 0;'>
                            <h3 style='margin-top:0; color:#155724;'>✅ Processed File Ready!</h3>
                            <p>The boundary data has been processed. Download the file to verify:</p>
                            <a href='{PROCESSED_DOWNLOAD_URL}' download='boundary_processed.xlsx' 
                               style='display:inline-block; padding:12px 24px; background:#28a745; color:white; 
                                      text-decoration:none; border-radius:6px; font-weight:bold;'>
                                📥 Download Processed File
                            </a>
                        </div>
                        """
                        display(HTML(download_link_html))
                        download_processed_btn.layout.visibility = 'visible'
                        check_status_btn.layout.visibility = 'hidden'
                elif status == 'inprogress':
                    print("\n⏳ Still processing... Check again in 10 seconds")
                else:
                    print(f"\n⚠️ Status: {status}")
            else:
                print("   No processing records found")
                
        except Exception as e:
            print(f"❌ Error: {str(e)[:200]}")

check_status_btn.on_click(on_check_status_clicked)

def on_download_processed_clicked(b):
    with output:
        if not PROCESSED_DOWNLOAD_URL:
            print("⚠️ No processed file URL available")
            return
        
        print(f"\n📥 Processed file URL ready!")
        print("   Click the green button above to download")

download_processed_btn.on_click(on_download_processed_clicked)

ui = widgets.VBox([
    widgets.HTML("<h3>📤 Step 2.2: Upload & Process</h3>"),
    widgets.HTML(f"<p style='background:#e7f3ff;padding:10px;border-radius:5px;'>"
                 f"<b>📋 Tenant:</b> {BOUNDARY_TENANT if BOUNDARY_TENANT else 'Not set'}<br>"
                 f"<b>🏛️ Hierarchy:</b> {BOUNDARY_HIERARCHY_TYPE if BOUNDARY_HIERARCHY_TYPE else 'Not set'}</p>"),
    widgets.HTML("<br><p><b>Select Filled Template:</b></p>"),
    boundary_file_upload,
    widgets.HTML("<p style='font-size:11px;color:#ff6600;'><i>⚠️ Make sure template is filled</i></p>"),
    widgets.HTML("<br>"),
    upload_process_btn,
    widgets.HTML("<br>"),
    check_status_btn,
    download_processed_btn,
    widgets.HTML("<br>"),
    output
])

display(ui)

   📤 PHASE 2.2: UPLOAD & PROCESS BOUNDARY DATA


---

# 🟪 PHASE 3: COMMON MASTER DATA

## 📋 Overview
In Phase 3, you will configure all common master data and PGR complaint configurations for your tenant.

## 🎯 What You'll Do:
1. **Enter Target Tenant ID** (e.g., `pg.citya` for city-level tenant)
2. **Upload Common Master Excel** with departments, designations, and complaint types
3. **System automatically creates:**
   - Department records in MDMS
   - Designation records in MDMS
   - Complaint types and sub-types
   - SLA configurations
   - Localization entries for all master data

## 📄 Template Required:
- **Common and Complaint Master.xlsx**

## 📊 Sheets in Template:
1. **Department Master** - List all departments (e.g., Public Works, Sanitation)
2. **Designation Master** - List all designations (e.g., Engineer, Supervisor)
3. **Complaint Type Master** - Main complaint categories
4. **Complaint SubType Master** - Sub-categories under each type

## ✅ What Gets Created:
- Departments with auto-generated codes (e.g., `DEPT_1`, `DEPT_2`)
- Designations with auto-generated codes (e.g., `DESIG_01`, `DESIG_02`)
- Complaint types with IDs and SLA hours
- Complaint sub-types linked to parent types
- Localization messages for all entries (en_IN, hi_IN)

## ⚠️ Important:
- Complete Phase 1 and Phase 2 before Phase 3
- Use city-level tenant ID (e.g., `pg.citya`, not just `pg`)
- Departments and Designations created here will be used in Phase 4 (Employee Creation)


---

## 📤 Step 3.1: Upload Common Master Excel

**Instructions:**
1. Enter the **Target Tenant ID** (e.g., `pg.citya`) in the text box
2. Click **Upload** button
3. Select **Common and Complaint Master.xlsx**
4. Click **Set Tenant & Upload** button

**What happens:**
- System reads all sheets (Departments, Designations, Complaint Types, Sub-Types)
- Validates data format
- Stores file for processing in next steps

**Expected Output:**
```
✅ Tenant set: pg.citya
✅ File uploaded: Common and Complaint Master.xlsx
```

💡 **Tip**: Use city-level tenant (e.g., `pg.citya`), not state-level (`pg`).


In [96]:
print("="*70)
print("        🎯  ENTER TARGET TENANT & UPLOAD COMMON MASTER")
print("="*70)
print()

# Tenant input widget
tenant_input = widgets.Text(
    value='',
    placeholder='e.g., pg, pg.citya, pg.cityb',
    description='🏛️ Tenant ID:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# File upload widget
common_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

# Combined upload button
upload_btn = widgets.Button(
    description='⬆️ Set Tenant & Upload Common Master',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

upload_output = widgets.Output()

def on_upload(b):
    global COMMON_MASTER_FILE, SELECTED_TENANT
    
    with upload_output:
        clear_output()
        
        # Validate tenant input
        if not tenant_input.value.strip():
            print("❌ Please enter a tenant ID")
            return
        
        # Validate file selection
        if not common_file_upload.value:
            print("❌ Please select Common Master Excel file")
            return
        
        # Set tenant
        SELECTED_TENANT = tenant_input.value.strip().lower()
        
        # Save file
        os.makedirs('upload', exist_ok=True)
        uploaded_file = common_file_upload.value[0]
        content = uploaded_file['content']
        filename = 'Common_Master.xlsx'
        
        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)
        
        print("="*70)
        print("  📋 VALIDATING FILE AGAINST MDMS SCHEMA...")
        print("="*70)
        
        # Validate the file against MDMS schema
        try:
            from mdms_validator import MDMSValidator
            mdms_validator = MDMSValidator(
                base_url=UPLOADER.base_url,
                auth_token=UPLOADER.auth_token,
                user_info=UPLOADER.user_info
            )
            result = mdms_validator.validate_excel_file(
                excel_file=upload_path,
                tenant_id=SELECTED_TENANT,
                schema_code='common.master'
            )
            
            if result['valid']:
                print("\n✅ MDMS VALIDATION PASSED!")
                print("   All required fields match MDMS schema.")
                COMMON_MASTER_FILE = upload_path
                
                print("\n" + "="*70)
                print("  ✅ FILE UPLOADED SUCCESSFULLY")
                print("="*70)
                print(f"\n🎯 Target Tenant: {SELECTED_TENANT}")
                print(f"\n📄 File: {upload_path}")
                print("\n➡️  Proceed to load and upload departments & designations")
                print("="*70)
            else:
                print("\n❌ MDMS VALIDATION FAILED!")
                print(f"   Found {len(result['errors'])} error(s):\n")
                
                # Show first 5 errors
                for i, error in enumerate(result['errors'][:5], 1):
                    print(f"   Error {i}:")
                    if 'row' in error:
                        print(f"      Sheet: {error.get('sheet', 'Unknown')}")
                        print(f"      Row: {error['row']}")
                        if 'column' in error:
                            print(f"      Column: {error['column']}")
                        if 'value' in error:
                            print(f"      Value: {error.get('value', 'N/A')}")
                    print(f"      Message: {error['message']}")
                    print()
                
                if len(result['errors']) > 5:
                    print(f"   ... and {len(result['errors']) - 5} more errors")
                
                print("\n⚠️  Please fix the errors in your Excel file and try again.")
                print("="*70)
                COMMON_MASTER_FILE = None
                SELECTED_TENANT = None
                
        except Exception as e:
            print(f"\n⚠️  MDMS Validation error: {str(e)}")
            print("   Proceeding without validation...")
            COMMON_MASTER_FILE = upload_path
            print("\n" + "="*70)
            print("  ✅ FILE UPLOADED SUCCESSFULLY")
            print("="*70)
            print(f"\n🎯 Target Tenant: {SELECTED_TENANT}")
            print(f"\n📄 File: {upload_path}")
            print("\n➡️  Proceed to load and upload departments & designations")
            print("="*70)

upload_btn.on_click(on_upload)

# Show uploaded tenants as reference (if available)
if UPLOADED_TENANTS:
    reference_info = widgets.HTML(
        f"<p style='color: #666; background-color: #f0f0f0; padding: 10px; border-radius: 5px;'>"
        f"<b>📋 Reference - Uploaded Tenants from Phase 1:</b><br>"
        f"{', '.join(UPLOADED_TENANTS)}</p>"
    )
else:
    reference_info = widgets.HTML("")

# Display
upload_box = widgets.VBox([
    widgets.HTML("<h3>🎯 Step 2.1: Enter Target Tenant & Upload Common Master</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Enter the tenant ID and upload the Common Master Excel file</i></p>"),
    reference_info,
    widgets.HTML("<br>"),
    tenant_input,
    widgets.HTML("<p style='font-size: 12px; color: #888;'><i>💡 You can upload to any tenant - it doesn't have to be from Phase 1</i></p>"),
    widgets.HTML("<br>"),
    widgets.HTML("<p style='color: #666;'><b>Select Common Master Excel File:</b></p>"),
    common_file_upload,
    widgets.HTML("<p style='font-size: 11px; color: #007bff;'><i>⚡ File will be validated against MDMS schema 'common.master'</i></p>"),
    widgets.HTML("<br>"),
    upload_btn,
    upload_output
])

display(upload_box)

        🎯  ENTER TARGET TENANT & UPLOAD COMMON MASTER



In [97]:
print("="*70)
print("[PHASE 2] LOADING DEPARTMENTS & DESIGNATIONS")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")


if not COMMON_MASTER_FILE:
    print("❌ Please upload Common Master Excel first!")
elif not SELECTED_TENANT:
    print("❌ Please select a tenant first!")
else:
    # Initialize reader
    reader = UnifiedExcelReader(COMMON_MASTER_FILE)
    uploader = UPLOADER
    
    # Read departments and designations (combined sheet)
    # Now also returns dept_name_to_code mapping for complaint types
    departments, designations, dept_loc, desig_loc, dept_name_to_code = reader.read_departments_designations(SELECTED_TENANT)
    
    # Create reverse mapping (code to name) for error Excel
    dept_code_to_name = {code: name for name, code in dept_name_to_code.items()}
    
    print(f"\n[INFO] Loaded {len(departments)} department(s)")
    for dept in departments:
        print(f"   - {dept['code']}: {dept['name']}")
    
    print(f"\n[INFO] Loaded {len(designations)} designation(s)")
    for desig in designations[:5]:
        print(f"   - {desig['code']}: {desig['name']} [{desig['department']}]")
    if len(designations) > 5:
        print(f"   ... and {len(designations) - 5} more")
    
    print(f"\n[INFO] Auto-generated {len(dept_loc) + len(desig_loc)} localization entries")
    print(f"\n[INFO] Created department name-to-code mapping: {dept_name_to_code}")
    
    # Store reverse mapping for error Excel
    uploader._dept_code_to_name = dept_code_to_name
    
    # Upload Departments (use combined sheet name for errors)
    result_dept = uploader.create_mdms_data(
        schema_code='common-masters.Department',
        data_list=clean_nans(departments),
        tenant=SELECTED_TENANT,
        sheet_name='Department_And_Designation'  # Combined sheet name
    )
    
    # Upload Designations (use same combined sheet name for errors)
    result_desig = uploader.create_mdms_data(
        schema_code='common-masters.Designation',
        data_list=clean_nans(designations),
        tenant=SELECTED_TENANT,
        sheet_name='Department And Desgination Mast', 
    excel_file=COMMON_MASTER_FILE  
    )
    
    # Upload Localization
    all_loc = dept_loc + desig_loc
    result_loc = uploader.create_localization_messages(
        localization_list=clean_nans(all_loc),
        tenant=SELECTED_TENANT,
        sheet_name='Dept_Desig_Localization'
    )
    
    # Summary
    if result_dept['failed'] == 0 and result_desig['failed'] == 0:
        print("\n✅ [SUCCESS] Departments & Designations uploaded successfully!")
        print("\n➡️  Proceed to upload Complaint Types")
    else:
        print("\n⚠️  [WARNING] Some items failed. Check errors/ folder.")

[PHASE 2] LOADING DEPARTMENTS & DESIGNATIONS
❌ Please upload Common Master Excel first!


## Step 2.3: Load and Upload Complaint Types

In [82]:
print("="*70)
print("[PHASE 2] LOADING COMPLAINT TYPES")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")


if not COMMON_MASTER_FILE:
    print("❌ Please upload Common Master Excel first!")
elif not SELECTED_TENANT:
    print("❌ Please select a tenant first!")
elif 'dept_name_to_code' not in locals():
    print("❌ Please run 'Load Departments & Designations' cell first!")
else:
    # Initialize reader
    reader = UnifiedExcelReader(COMMON_MASTER_FILE)
    uploader = UPLOADER
    
    # Read complaint types - pass dept_name_to_code mapping so it uses department codes
    complaint_types, ct_loc = reader.read_complaint_types(SELECTED_TENANT, dept_name_to_code)
    
    print(f"\n[INFO] Loaded {len(complaint_types)} complaint type(s)")
    for ct in complaint_types[:10]:
        dept_info = f" [Dept: {ct.get('department', 'N/A')}]" if ct.get('department') else ""
        print(f"   - {ct['serviceCode']}: {ct['name']}{dept_info}")
    if len(complaint_types) > 10:
        print(f"   ... and {len(complaint_types) - 10} more")
    
    print(f"\n[INFO] Auto-generated {len(ct_loc)} localization entries")
    
    # Upload Complaint Types (pass reverse mapping for error Excel)
    uploader._dept_code_to_name = dept_code_to_name
    result_ct = uploader.create_mdms_data(
        schema_code='RAINMAKER-PGR.ServiceDefs',
        data_list=clean_nans(complaint_types),
        tenant=SELECTED_TENANT,
        sheet_name='Complaint Type Master',
          excel_file=COMMON_MASTER_FILE  
    )
    
    # Upload Localization
    result_loc = uploader.create_localization_messages(
        localization_list=clean_nans(ct_loc),
        tenant=SELECTED_TENANT,
        sheet_name='ComplaintTypes_Localization'
    )
    
    # Summary
    if result_ct['failed'] == 0:
        print("\n✅ [SUCCESS] Complaint Types uploaded successfully!")
        print("\n🎉 PHASE 2 COMPLETED!")
    else:
        print("\n⚠️  [WARNING] Some complaint types failed. Check errors/ folder.")

[PHASE 2] LOADING COMPLAINT TYPES

[INFO] Loaded 1 complaint type(s)
   - TabBroken: tab broken [Dept: DEPT_1]

[INFO] Auto-generated 2 localization entries

[UPLOADING] RAINMAKER-PGR.ServiceDefs
   Tenant: pg
   Records: 1
   API URL: https://unified-qa.digit.org/mdms-v2/v2/_create/{schema_code}
   [FAILED] [1/1] TabBroken (HTTP 401)
   ERROR: You are not authorized to access this resource - You are not authorized to access this resource
[SUMMARY] Created: 0
[SUMMARY] Already Exists: 0
[SUMMARY] Failed: 1

📝 Updating Excel file: upload/Common_Master.xlsx
   Sheet: Complaint Type Master
   ✅ Status columns updated successfully!
   📊 Updated 1 rows

[UPLOADING] Localization Messages
   Tenant: pg
   Total Messages: 2
   API URL: https://unified-qa.digit.org/localization/localization/messages/v1/_upsert

   Found 1 locales: en_IN
   [FAILED] Locale: en_IN (HTTP 401)
   ERROR: You are not authorized to access this resource - You are not authorized to access this resource
[SUMMARY] Created

---

# 🟦 PHASE 4: EMPLOYEE BULK CREATION

In Phase 4, you will bulk create employees for your tenant.

## 📋 Overview:
1. **Generate Dynamic Template** - Creates Excel with dropdowns pre-filled from MDMS
2. **Fill Employee Data** - Use the template to add employee details
3. **Upload Excel** - Upload the filled template
4. **Auto Role Validation** - System checks & creates missing roles in MDMS
5. **Bulk Create** - Creates all employees via HRMS API
6. **Status Tracking** - Excel updated with color-coded status (🟢 SUCCESS, 🟡 EXISTS, 🔴 FAILED)

## 🎯 Key Features:
- ✅ **Name-based Excel** - Use department/designation/role NAMES, system converts to codes
- ✅ **Date pickers** - Excel date inputs instead of timestamps
- ✅ **README - Roles sheet** - Copy-paste ready role names
- ✅ **Auto role creation** - Missing roles automatically created in MDMS
- ✅ **Error handling** - Status appended to each row with details

## 📖 Template Sheets:
1. **Instructions** - Complete usage guide
2. **README - Roles** - All available roles to copy from
3. **Employee Master** - Main data entry sheet
4. **Hidden reference sheets** - Departments, Designations, Roles, Boundaries


---

## 🎨 Step 4.1: Generate Dynamic Employee Template

**Instructions:**
1. Enter the **Target Tenant ID** (e.g., `pg.citya`) in the text box
2. Click **🎨 Generate Dynamic Template** button
3. Wait for template generation (may take 10-20 seconds)

**What happens:**
- System fetches **Departments** from MDMS → Creates dropdown
- System fetches **Designations** from MDMS → Creates dropdown
- System fetches **Roles** from MDMS → Creates README sheet
- System fetches **Boundaries** from Boundary Service → Creates dropdown
- Creates **Employee Master** sheet with data validation
- Creates **Instructions** sheet with usage guide
- Creates **README - Roles** sheet with all available roles

**Generated File Location:**
```
templates/Employee_Master_Dynamic_{tenant}.xlsx
```

**Template Contains:**
- ✅ All dropdowns pre-filled from MDMS
- ✅ Excel date pickers for date fields
- ✅ README - Roles sheet with copy-paste ready role names
- ✅ Instructions sheet with complete guidance
- ✅ Sample row with default values

💡 **Tip**: This template is customized for your tenant with real data from MDMS!


In [83]:
print("="*70)
print("        🎨  GENERATE DYNAMIC EMPLOYEE TEMPLATE FROM MDMS")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")

print()

# Tenant input for template generation
template_tenant_input = widgets.Text(
    value='',
    placeholder='e.g., pg.citya, pb.amritsar',
    description='🏛️ Tenant ID:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# Generate button
generate_employee_template_btn = widgets.Button(
    description='🎨 Generate Dynamic Template',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px')
)

template_gen_output = widgets.Output()

GENERATED_EMP_TEMPLATE_PATH = None

def on_generate_emp_template(b):
    global GENERATED_EMP_TEMPLATE_PATH
    
    with template_gen_output:
        clear_output()
        
        if not template_tenant_input.value.strip():
            print("❌ Please enter a tenant ID")
            return
        
        tenant = template_tenant_input.value.strip().lower()
        
        print(f"🔄 Generating dynamic template for tenant: {tenant}")
        print("   Fetching data from MDMS...\n")
        
        uploader = UPLOADER
        
        try:
            template_path = uploader.generate_employee_template(tenant)
            GENERATED_EMP_TEMPLATE_PATH = template_path
            
            download_link_html = f"""
            <div style='background:#d4edda; padding:15px; border-radius:8px; margin:10px 0;'>
                <h3 style='margin-top:0; color:#155724;'>✅ Dynamic Template Generated!</h3>
                <p><b>File:</b> {template_path}</p>
                <a href='{template_path}' download='Employee_Master_Dynamic.xlsx' 
                   style='display:inline-block; padding:12px 24px; background:#28a745; color:white; 
                          text-decoration:none; border-radius:6px; font-weight:bold; margin:10px 0;'>
                    📥 Download Template
                </a>
                <p style='margin-top:15px;'><b>✨ Features:</b></p>
                <ul style='margin:5px 0;'>
                    <li>✅ Departments fetched from MDMS</li>
                    <li>✅ Designations fetched from MDMS</li>
                    <li>✅ Roles fetched from User Service</li>
                    <li>✅ Boundaries fetched from Boundary Service</li>
                    <li>✅ 8 pre-filled dropdowns</li>
                    <li>✅ Employee code auto-generated</li>
                    <li>✅ Sample row included</li>
                </ul>
            </div>
            """
            display(HTML(download_link_html))
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")

generate_employee_template_btn.on_click(on_generate_emp_template)

if 'UPLOADED_TENANTS' in locals() and UPLOADED_TENANTS:
    emp_ref = widgets.HTML(
        f"<p style='background:#f0f0f0; padding:10px; border-radius:5px;'>"
        f"<b>📋 Tenants from Phase 1:</b> {', '.join(UPLOADED_TENANTS)}</p>"
    )
else:
    emp_ref = widgets.HTML("")

display(widgets.VBox([
    widgets.HTML("<h3>🎨 Step 4.0: Generate Dynamic Template</h3>"),
    widgets.HTML("<div style='background:#fff3cd;padding:12px;border-radius:5px;'>"
                 "<b>💡 Recommended:</b> Generate template with live MDMS data - dropdowns auto-filled!</div>"),
    emp_ref,
    widgets.HTML("<br>"),
    template_tenant_input,
    widgets.HTML("<br>"),
    generate_employee_template_btn,
    template_gen_output
]))

        🎨  GENERATE DYNAMIC EMPLOYEE TEMPLATE FROM MDMS



---

### 📝 NEXT STEPS: Fill the Template

**After generating the template:**

1. **Open the Excel file** from the `templates/` folder
2. **Read the 'Instructions' sheet** for detailed guidance
3. **Check 'README - Roles' sheet** for available role names
4. **Fill the 'Employee Master' sheet** with employee data:
   - Use NAMES for departments, designations, and roles (NOT codes)
   - Use Excel date picker for dates
   - For multiple roles: copy from README and join with commas (e.g., `Employee,PGR Viewer,PGR Admin`)
5. **Delete the sample row** before uploading
6. **Save the file** and proceed to Step 4.1 below

💡 **TIP**: The 'README - Roles' sheet has all available roles with descriptions - just copy the role names you need!


In [84]:
print("="*70)
print("        👥  PHASE 4: UPLOAD EMPLOYEE MASTER")
print("="*70)
print()

# Global variable for employee file
EMPLOYEE_MASTER_FILE = None
EMPLOYEE_TENANT = None

# Tenant input widget
employee_tenant_input = widgets.Text(
    value='',
    placeholder='e.g., pg.citya, pb.amritsar',
    description='🏛️ Tenant ID:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# File upload widget
employee_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

# Upload button
employee_upload_btn = widgets.Button(
    description='⬆️ Set Tenant & Upload Employee Master',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

employee_upload_output = widgets.Output()

def on_employee_upload(b):
    global EMPLOYEE_MASTER_FILE, EMPLOYEE_TENANT
    
    with employee_upload_output:
        clear_output()
        
        # Validate tenant input
        if not employee_tenant_input.value.strip():
            print("❌ Please enter a tenant ID")
            return
        
        # Validate file selection
        if not employee_file_upload.value:
            print("❌ Please select Employee Master Excel file")
            return
        
        # Set tenant
        EMPLOYEE_TENANT = employee_tenant_input.value.strip().lower()
        
        # Save file
        os.makedirs('upload', exist_ok=True)
        uploaded_file = employee_file_upload.value[0]
        content = uploaded_file['content']
        filename = 'Employee_Master.xlsx'
        
        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)
        
        EMPLOYEE_MASTER_FILE = upload_path
        
        print("="*70)
        print("  ✅ FILE UPLOADED SUCCESSFULLY")
        print("="*70)
        print(f"\n🎯 Target Tenant: {EMPLOYEE_TENANT}")
        print(f"\n📄 File: {upload_path}")
        print("\n➡️  Proceed to bulk create employees")
        print("="*70)

employee_upload_btn.on_click(on_employee_upload)

# Show uploaded tenants as reference (if available)
if UPLOADED_TENANTS:
    reference_info = widgets.HTML(
        f"<p style='color: #666; background-color: #f0f0f0; padding: 10px; border-radius: 5px;'>"
        f"<b>📋 Reference - Uploaded Tenants from Phase 1:</b><br>"
        f"{', '.join(UPLOADED_TENANTS)}</p>"
    )
else:
    reference_info = widgets.HTML("")

# Display
employee_upload_box = widgets.VBox([
    widgets.HTML("<h3>👥 Step 4.1: Upload Employee Master</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Upload Employee Master Excel to bulk create employees</i></p>"),
    reference_info,
    widgets.HTML("<br>"),
    employee_tenant_input,
    widgets.HTML("<p style='font-size: 12px; color: #888;'><i>💡 Employees will be created for this tenant</i></p>"),
    widgets.HTML("<br>"),
    widgets.HTML("<p style='color: #666;'><b>Select Employee Master Excel File:</b></p>"),
    employee_file_upload,
    widgets.HTML("<br>"),
    employee_upload_btn,
    employee_upload_output
])

display(employee_upload_box)

        👥  PHASE 4: UPLOAD EMPLOYEE MASTER



---

### ⚡ Step 4.2: Bulk Create Employees

**What happens when you run the cell below:**

#### 🔐 Phase 1: Role Validation (Automatic)
- System checks if all required roles exist in MDMS
- If roles are missing, they are **automatically created** from default-data-handler configuration
- Ensures HRMS validation won't fail due to missing roles

#### 👥 Phase 2: Employee Creation
- Converts all NAMES to CODES internally (departments, designations, roles)
- Converts Excel dates to timestamps
- Auto-generates employee codes from user names
- Creates jurisdiction from boundary + roles
- Calls HRMS API for each employee

#### 📊 Phase 3: Status Tracking
- Updates your Excel file with 3 new columns:
  - `_STATUS` (SUCCESS/EXISTS/FAILED)
  - `_STATUS_CODE` (HTTP code)
  - `_ERROR_MESSAGE` (error details)
- Color codes rows:
  - 🟢 **GREEN** = Successfully created
  - 🟡 **YELLOW** = Already exists (duplicate)
  - 🔴 **RED** = Failed (see error message)

#### 📈 Final Summary
- Shows count of created/exists/failed employees
- Opens updated Excel file for review

**Ready? Run the cell below to start bulk creation! ⬇️**


In [87]:
print("="*70)
print("[PHASE 4] BULK CREATING EMPLOYEES")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")


if not EMPLOYEE_MASTER_FILE:
    print("❌ Please upload Employee Master Excel first!")
elif not EMPLOYEE_TENANT:
    print("❌ Please set target tenant first!")
else:
    # Initialize reader and uploader
    reader = UnifiedExcelReader(EMPLOYEE_MASTER_FILE)
    uploader = UPLOADER
    
    # Read employees using bulk method
    employees = reader.read_employees_bulk(EMPLOYEE_TENANT)
    
    print(f"\n[INFO] Loaded {len(employees)} employee(s) from Excel")
    for emp in employees[:5]:
        dept = emp['assignments'][0]['department']
        desig = emp['assignments'][0]['designation']
        roles = [r['code'] for r in emp['user']['roles']]
        print(f"   - {emp['code']}: {emp['user']['name']} [{dept}/{desig}] Roles: {', '.join(roles)}")
    if len(employees) > 5:
        print(f"   ... and {len(employees) - 5} more")
    
    # Create employees
    result_employees = uploader.create_employees(
        employee_list=clean_nans(employees),
        tenant=EMPLOYEE_TENANT,
        sheet_name='Employee Master',
        excel_file=EMPLOYEE_MASTER_FILE
    )
    
    # Summary
    if result_employees['failed'] == 0:
        print("\n✅ [SUCCESS] Employees created successfully!")
        print("\n🎉 PHASE 4 COMPLETED!")
    else:
        print("\n⚠️  [WARNING] Some employees failed. Check errors/ folder or status columns in Excel.")

[PHASE 4] BULK CREATING EMPLOYEES


ValueError: base_url is required. Please provide gateway URL.

In [86]:
from IPython.display import display, HTML, FileLink
from datetime import datetime
import os

# Collect results
summary_data = []

if 'result_tenants' in locals():
    summary_data.append({
        'module': 'Tenants',
        'created': result_tenants.get('created', 0),
        'exists': result_tenants.get('exists', 0),
        'failed': result_tenants.get('failed', 0),
        'excel_file': TENANT_FILE if 'TENANT_FILE' in locals() else None,
        'sheet': 'Tenant Info'
    })

if 'result_dept' in locals():
    summary_data.append({
        'module': 'Departments',
        'created': result_dept.get('created', 0),
        'exists': result_dept.get('exists', 0),
        'failed': result_dept.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Department And Desgination Mast'
    })

if 'result_desig' in locals():
    summary_data.append({
        'module': 'Designations',
        'created': result_desig.get('created', 0),
        'exists': result_desig.get('exists', 0),
        'failed': result_desig.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Department And Desgination Mast'
    })

if 'result_ct' in locals():
    summary_data.append({
        'module': 'Complaint Types',
        'created': result_ct.get('created', 0),
        'exists': result_ct.get('exists', 0),
        'failed': result_ct.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Complaint Type Master'
    })

if 'result_employees' in locals():
    summary_data.append({
        'module': 'Employees (HRMS)',
        'created': result_employees.get('created', 0),
        'exists': result_employees.get('exists', 0),
        'failed': result_employees.get('failed', 0),
        'excel_file': EMPLOYEE_MASTER_FILE if 'EMPLOYEE_MASTER_FILE' in locals() else None,
        'sheet': 'Employee Master'
    })

# Calculate totals
total_created = sum(item['created'] for item in summary_data)
total_exists = sum(item['exists'] for item in summary_data)
total_failed = sum(item['failed'] for item in summary_data)
total_records = total_created + total_exists + total_failed

# Build HTML table
table_rows = []
for item in summary_data:
    total_for_module = item['created'] + item['exists'] + item['failed']
    table_rows.append(f"""
        <tr>
            <td style="padding: 8px; border: 1px solid #ddd;">{item['module']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: green; font-weight: bold;">{item['created']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: orange; font-weight: bold;">{item['exists']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: red; font-weight: bold;">{item['failed']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; font-weight: bold;">{total_for_module}</td>
        </tr>
    """)

timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Build download links section for updated Excel files
updated_files = {}
for item in summary_data:
    if item['excel_file'] and os.path.exists(item['excel_file']):
        file_path = item['excel_file']
        file_name = os.path.basename(file_path)
        if file_path not in updated_files:
            updated_files[file_path] = {
                'name': file_name,
                'sheets': [],
                'modules': [],
                'has_errors': False,
                'has_success': False
            }
        updated_files[file_path]['sheets'].append(item['sheet'])
        updated_files[file_path]['modules'].append(item['module'])
        if item['failed'] > 0:
            updated_files[file_path]['has_errors'] = True
        if item['created'] > 0 or item['exists'] > 0:
            updated_files[file_path]['has_success'] = True

# Generate download links HTML
download_links_html = ""

# Check for legacy error file (from old approach)
legacy_error_file = 'errors/FAILED_RECORDS.xlsx'
has_legacy_errors = os.path.exists(legacy_error_file)

if updated_files:
    download_links_html = """
    <div style="margin: 20px 0; padding: 15px; background-color: #e7f3ff; border-left: 4px solid #007bff; border-radius: 5px;">
        <h3 style="margin-top: 0; color: #004085;">📥 Updated Excel Files with Status Columns</h3>
        <p style="color: #004085; margin-bottom: 15px;">
            The following Excel files have been updated with <b>_STATUS</b>, <b>_STATUS_CODE</b>, and <b>_ERROR_MESSAGE</b> columns:
        </p>
    """
    
    for file_path, file_info in updated_files.items():
        # Determine status badge
        if file_info['has_errors']:
            status_badge = '<span style="background-color: #dc3545; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">⚠️ HAS ERRORS</span>'
        else:
            status_badge = '<span style="background-color: #28a745; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">✅ ALL SUCCESS</span>'
        
        modules_list = ', '.join(file_info['modules'])
        
        download_links_html += f"""
        <div style="margin-bottom: 15px; padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #ddd;">
            <div style="display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap;">
                <div style="flex: 1; min-width: 300px;">
                    <strong style="color: #007bff;">📄 {file_info['name']}</strong> {status_badge}
                    <br>
                    <span style="font-size: 12px; color: #666;">Modules: {modules_list}</span>
                </div>
                <a href="{file_path}" download="{file_info['name']}" 
                   style="display: inline-block; padding: 8px 16px; background-color: #007bff; color: white; 
                          text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px; margin-top: 5px;">
                    ⬇️ Download Updated File
                </a>
            </div>
        </div>
        """
    
    download_links_html += """
        <p style="color: #004085; font-size: 12px; margin-top: 15px; border-top: 1px solid #bee5eb; padding-top: 10px;">
            <b>💡 How to use:</b>
        </p>
        <ul style="font-size: 12px; color: #004085; margin: 5px 0;">
            <li><span style="color: green; font-weight: bold;">🟢 GREEN rows (SUCCESS):</span> Successfully created in MDMS/HRMS</li>
            <li><span style="color: orange; font-weight: bold;">🟡 YELLOW rows (EXISTS):</span> Already exist in system (duplicates)</li>
            <li><span style="color: red; font-weight: bold;">🔴 RED rows (FAILED):</span> Failed to create - check _ERROR_MESSAGE column for details</li>
        </ul>
        <p style="font-size: 12px; color: #004085; margin-top: 10px;">
            <b>🔒 Note:</b> Status columns are <b>protected</b> and cannot be edited. Fix data in other columns and re-upload if needed.
        </p>
    </div>
    """

# Add legacy error file section if it exists
if has_legacy_errors and total_failed > 0:
    download_links_html += f"""
    <div style="margin: 20px 0; padding: 15px; background-color: #fff3cd; border-left: 4px solid #ffc107; border-radius: 5px;">
        <h3 style="margin-top: 0; color: #856404;">⚠️ Legacy Error File Detected</h3>
        <p style="color: #856404; margin-bottom: 10px;">
            An error file was generated from a previous upload approach. You can download it below:
        </p>
        <div style="padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #ddd;">
            <div style="display: flex; justify-content: space-between; align-items: center;">
                <div>
                    <strong style="color: #856404;">📄 FAILED_RECORDS.xlsx</strong>
                    <br>
                    <span style="font-size: 12px; color: #666;">Contains {total_failed} failed record(s)</span>
                </div>
                <a href="{legacy_error_file}" download="FAILED_RECORDS.xlsx" 
                   style="display: inline-block; padding: 8px 16px; background-color: #dc3545; color: white; 
                          text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px;">
                    ⬇️ Download Error File
                </a>
            </div>
        </div>
        <p style="color: #856404; font-size: 12px; margin-top: 10px;">
            <b>💡 Tip:</b> Use the updated Excel files above instead - they have status columns directly in your original templates!
        </p>
    </div>
    """

html_content = f"""
<div style="font-family: Arial, sans-serif; padding: 20px; border: 2px solid #007bff; border-radius: 10px; background-color: #f8f9fa;">
    <h2 style="color: #007bff; margin-top: 0;">📊 Data Upload Summary Report</h2>
    <p style="color: #666; margin-bottom: 20px;">Generated: {timestamp}</p>
    
    <table style="width: 100%; border-collapse: collapse; margin-bottom: 20px; background-color: white;">
        <thead>
            <tr style="background-color: #007bff; color: white;">
                <th style="padding: 12px; border: 1px solid #ddd; text-align: left;">Module</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">✅ Created</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">⚠️ Already Exists</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">❌ Failed</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">📊 Total</th>
            </tr>
        </thead>
        <tbody>
            {''.join(table_rows)}
            <tr style="background-color: #e9ecef; font-weight: bold;">
                <td style="padding: 12px; border: 1px solid #ddd;">TOTAL</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: green;">{total_created}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: orange;">{total_exists}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: red;">{total_failed}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center;">{total_records}</td>
            </tr>
        </tbody>
    </table>
    
    <div style="display: flex; justify-content: space-around; margin: 20px 0; flex-wrap: wrap;">
        <div style="text-align: center; padding: 15px; background-color: #d4edda; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #155724;">{total_created}</div>
            <div style="color: #155724;">Created</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #fff3cd; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #856404;">{total_exists}</div>
            <div style="color: #856404;">Already Exists</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #f8d7da; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #721c24;">{total_failed}</div>
            <div style="color: #721c24;">Failed</div>
        </div>
    </div>
    
    {download_links_html}
</div>
"""

display(HTML(html_content))

print("\n🎉 DATA UPLOAD COMPLETED!")
if updated_files:
    print(f"\n📥 {len(updated_files)} Excel file(s) updated with status columns")
    for file_path in updated_files.keys():
        print(f"   • {os.path.basename(file_path)}")

Module,✅ Created,⚠️ Already Exists,❌ Failed,📊 Total
Tenants,0,0,2,2
Departments,0,0,2,2
Designations,0,0,2,2
Complaint Types,0,0,1,1
TOTAL,0,0,7,7



🎉 DATA UPLOAD COMPLETED!

📥 2 Excel file(s) updated with status columns
   • Tenant_Master.xlsx
   • Common_Master.xlsx
